In [36]:
# ------- NOTEBOOK CONFIGURATION -------

model_name = "xlm-roberta-base" # "roberta-base" or "xlm-roberta-base"
pre_finetuned = True
finetuning_language = "en-fr" # "en" or "fr" or "en-fr" or "de" or "es" or "it" # only "en" implemented so far
dataset_name = "loria" # 'dihana' (Spanish), 'ilisten' (Italian), 'loria' (French), 'maptask' (English) or 'vm2' (German) 
last_layer_only = True

# ------ DO NOT CHANGE UNDER HERE ------

import os
import re

n_train = 1986 # number of training examples in the Ilisten dataset (minimum of all five datasets)
n_test  =  971 # number of     test examples in the Ilisten dataset (minimum of all five datasets)

if model_name=="roberta-base" and dataset_name != "maptask":
    print("Warning: RoBERTa is only trainable on the Maptask dataset.")

if pre_finetuned:
    model_path  = './results/' + model_name + '--open-subtitle-' + finetuning_language + "--last-3-layers"
    saving_path =  model_path + '--' + dataset_name + '--' + ('last-layer-only' if last_layer_only else 'full')
    checkpoint_steps = [ int(re.search(r'\d+$', d).group()) for d in os.listdir(model_path) ]
    latest_checkpoint = max(checkpoint_steps)
    model_path = model_path + f'/checkpoint-{latest_checkpoint}'
else :
    model_path = model_name
    saving_path = './results/' + model_name + '--' + dataset_name + '--' + ('last-layer-only' if last_layer_only else 'full')

# find directories ending with numbers using regex
print(f"Loading pre-finetuned on Open Subtitle model from {model_path}")
print(f"Writing to {saving_path}")

Loading pre-finetuned on Open Subtitle model from ./results/xlm-roberta-base--open-subtitle-en-fr--last-3-layers/checkpoint-37000
Writing to ./results/xlm-roberta-base--open-subtitle-en-fr--last-3-layers--loria--last-layer-only


In [37]:
! pip3 install torch transformers ipywidgets datasets accelerate evaluate wheel scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 20.8 MB/s eta 0:00:0000:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [3]:
import numpy as np

In [37]:
# Based on roBERTa and using the transformers library
# and the https://huggingface.co/datasets/silicone dataset
# train a classifier of each "utterance" into the categories 0 to 3

# Ideally, the classification should depend on the dialog context
# using the Dialogue_ID and Idx variables
# respectively giving the dialogue identifier and the utterance order.

# Inspired by https://huggingface.co/docs/transformers/training

import datasets
dataset = datasets.load_dataset('miam', dataset_name)
n_intent_classes = len(dataset['train'].info.features['Label'].names)

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=n_intent_classes)

Found cached dataset miam (/Users/katossky/.cache/huggingface/datasets/miam/loria/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c)


  0%|          | 0/3 [00:00<?, ?it/s]

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/katossky/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading file sent

In [38]:
def tokenize_function(doc):
    return tokenizer(doc["Utterance"], padding="max_length", truncation=True)

In [39]:
columns_to_remove = np.setdiff1d(dataset['train'].column_names, ['Label'])
dataset_train = dataset['train']\
    .shuffle(seed=42).select(range(n_train))\
    .rename_column("Label", "label")\
    .map(tokenize_function, batched=True)\
    .remove_columns(columns_to_remove)

Loading cached shuffled indices for dataset at /Users/katossky/.cache/huggingface/datasets/miam/loria/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c/cache-02f63c95f29d78ad.arrow
Loading cached processed dataset at /Users/katossky/.cache/huggingface/datasets/miam/loria/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c/cache-378c32391a03c315.arrow


In [40]:
columns_to_remove = np.setdiff1d(dataset['test'].column_names, ['Label'])
dataset_test = dataset['test']\
    .shuffle(seed=42).select(range(n_test))\
    .rename_column("Label", "label")\
    .map(tokenize_function, batched=True)\
    .remove_columns(columns_to_remove)

Loading cached shuffled indices for dataset at /Users/katossky/.cache/huggingface/datasets/miam/loria/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c/cache-7388fd0845b6cec9.arrow
Loading cached processed dataset at /Users/katossky/.cache/huggingface/datasets/miam/loria/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c/cache-cd5cbf4fc11f84a3.arrow


In [41]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [42]:
# freeze the model but the last layer
if last_layer_only:
    for param in model.roberta.parameters():
        param.requires_grad = False

In [91]:
# count the number of trainable parameters
# sum(p.numel() for p in model.parameters() if p.requires_grad)

# model.classifier
# the head consists in two linear layers
# the first has 768 features in and 768 out
# the second has 768 features in and 4 out
# this is a total of 768 * 768 + 768 + 768 * 4 + 4 = 593668

# (model.classifier.dense.in_features + 1) * model.classifier.dense.out_features +\
# model.classifier.dense.out_features+1) * n_intent_classes
# yeaaaaah! this matches the number of trainable parameters

In [43]:
# train the model

# seems like worth it to test metaparameters
# in particular, I get no logging, and that
# may be due to the batch seize not beeing
# a multiple of gradient_accumulation_steps

training_args = TrainingArguments(
    seed=42,                         # random seed for initialization
    output_dir=saving_path,          # output directory
    evaluation_strategy="steps",     # evaluation strategy to adopt during training
    eval_steps=100,                  # evaluation step
    logging_steps=100,               # log step
    optim="adamw_torch",             # optimizer
    learning_rate=1e-3,              # learning rate
    weight_decay=0.01,               # strength of weight decay
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=50,                 # number of warmup steps for learning rate scheduler
    save_strategy="epoch",           # strategy to adopt when saving checkpoints
    use_mps_device=True,             # use the new Apple M1 chip
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [44]:
trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = dataset_train,         # training dataset
    eval_dataset  = dataset_test,          # evaluation dataset
    compute_metrics = compute_metrics,
)

In [45]:
trainer.train()

***** Running training *****
  Num examples = 1986
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 614431


  0%|          | 0/375 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 971
  Batch size = 64


{'loss': 2.4719, 'learning_rate': 0.0008461538461538462, 'epoch': 0.8}


  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 1.6380480527877808, 'eval_accuracy': 0.6220391349124614, 'eval_runtime': 52.2741, 'eval_samples_per_second': 18.575, 'eval_steps_per_second': 0.306, 'epoch': 2.4}


Saving model checkpoint to ./results/xlm-roberta-base--open-subtitle-en-fr--last-3-layers--loria--last-layer-only/checkpoint-375
Configuration saved in ./results/xlm-roberta-base--open-subtitle-en-fr--last-3-layers--loria--last-layer-only/checkpoint-375/config.json
Model weights saved in ./results/xlm-roberta-base--open-subtitle-en-fr--last-3-layers--loria--last-layer-only/checkpoint-375/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 397.6146, 'train_samples_per_second': 14.984, 'train_steps_per_second': 0.943, 'train_loss': 1.975560567220052, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.975560567220052, metrics={'train_runtime': 397.6146, 'train_samples_per_second': 14.984, 'train_steps_per_second': 0.943, 'train_loss': 1.975560567220052, 'epoch': 3.0})